# Tables and Units

In [ ]:
import numpy as np
import pandas as pd

from astropy import units as u
from astropy import constants as const

In [ ]:
comet_table = pd.read_csv('./Data/Comets.csv')

In [ ]:
comet_table

## `DataFrames` and units - A (not so good) example

* Use `.to_numpy()` to pull data out of `DataFrame`
* Then add the units

In [ ]:
semi_major = comet_table['Semi_Major_AU'].to_numpy()

semi_major

In [ ]:
semi_major = comet_table['Semi_Major_AU'].to_numpy() * u.AU

semi_major

In [ ]:
semi_major.to(u.km)

#### Since `DataFrames` do not have units - you need to use `.value` to add a column 

In [ ]:
comet_table['Semi_Major_km'] = semi_major.to(u.km).value

In [ ]:
comet_table

#### Pull out column and assign units everytime you want to use them.

* Even with dimensionless units (like `Eccentricity`).

$$\large
\mathrm{Perihelion\  Distance} = \mathrm{semi\ major\ axis} \times (1.0 - \mathrm{eccentricity})
$$

In [ ]:
def find_perihelion(semi_major, eccentricity):
    result = semi_major * (1.0 - eccentricity)
    return result

In [ ]:
my_semi_major = comet_table['Semi_Major_AU'].to_numpy() * u.AU

my_semi_major

In [ ]:
my_ecc = comet_table['Eccentricity'].to_numpy() * u.dimensionless_unscaled

my_ecc

In [ ]:
perihelion_AU = find_perihelion(my_semi_major, my_ecc)

perihelion_AU

In [ ]:
comet_table['Perihelion_AU'] = perihelion_AU.value

In [ ]:
comet_table

##### Save `comet_table` to a file (`.csv`)

In [ ]:
comet_table.to_csv('./Data/Comet_DataFrame.csv', index=False)

---

## `DataFrames` and units

 * `DatFrames` and units do not play together well
 * Using a `DataFrame` and units requires you to:
   * Pull out column and assign units everytime you want to use them.
     * `comet_table['Semi_Major_AU'].to_numpy() * u.AU`
   * The to save your results, without units, back to the table
     * `comet_table['Perihelion_AU'] = perihelion_AU.value`
     
## If you want to use units in a table, `DataFrames` are not your best option. What should you use?

---

# Astropy `QTable`

* A `QTable` = a table with units!
* Does not have the huge number of `.methods` of a `DataFrame`
* **Only** used by Astronomers
* Can be easily converted to a `DataFrame`

In [ ]:
from astropy.table import QTable

In [ ]:
comet_table = QTable.read('./Data/Comets.csv', format='ascii.csv')

In [ ]:
comet_table

In [ ]:
print(comet_table)

### Adding a unit to a column

In [ ]:
comet_table['Semi_Major_AU'].unit = u.AU

In [ ]:
comet_table

In [ ]:
comet_table['Semi_Major_AU']

In [ ]:
comet_table['Semi_Major_AU'].to(u.km)

In [ ]:
comet_table['Semi_Major_AU'].unit

### Functions and Tables

$$\large
\mathrm{Orbital\ Velocity} = \sqrt{GM_{\odot}\over d} 
$$

In [ ]:
def find_orbit_v(my_distance):
    result = np.sqrt(const.G * const.M_sun / my_distance)
    return result.decompose()

In [ ]:
find_orbit_v(comet_table['Semi_Major_AU'])

In [ ]:
comet_table['Oribtal_Velocity'] = find_orbit_v(comet_table['Semi_Major_AU'])

In [ ]:
comet_table

In [ ]:
comet_table['Oribtal_Velocity'].to(u.km / u.s)

In [ ]:
comet_table['Oribtal_Velocity'].info.format = '.2f'

In [ ]:
comet_table

In [ ]:
comet_table.info()

In [ ]:
for row in comet_table:
    output = f"The comet {row['Name']:9} has an orbital velocity of {row['Oribtal_Velocity']:.2f}"
    print(output)

## `QTable` manipulation and modification

* Does not have the huge number of `.methods` of a `DataFrame`
* Can do most 'obvious' stuff: slices, sorts, filtering, etc...
* Documentation: [Astropy Table Modifications](https://het.as.utexas.edu/HET/Software/Astropy-1.0/table/modify_table.html)

In [ ]:
comet_table

In [ ]:
comet_table['Name'][0]

In [ ]:
comet_table[0:2]

In [ ]:
comet_table[comet_table['Eccentricity'] < 0.8]

In [ ]:
comet_table[comet_table['Eccentricity'] < 0.8]['Name'][-1]

In [ ]:
comet_table.rename_column('Name', 'Comet Name')

In [ ]:
comet_table

In [ ]:
comet_table.sort('Oribtal_Velocity')

In [ ]:
comet_table

In [ ]:
comet_table['Comet Name'][0]

In [ ]:
comet_table.sort('Oribtal_Velocity', reverse=True)

comet_table

#### Comet with the 3rd fastest orbital velocity?

In [ ]:
comet_table[2]

In [ ]:
comet_table[2]['Oribtal_Velocity'].value

In [ ]:
comet_table[2]['Oribtal_Velocity'].unit

### Can save `QTables` with all the units info intact (`.ecsv`).

In [ ]:
comet_table.write('./Data/Comet_QTable.ecsv', format='ascii.ecsv', overwrite=True)

In [ ]:
my_new_table = QTable.read('./Data/Comet_QTable.ecsv', format='ascii.ecsv')

In [ ]:
my_new_table

---

### Can convert astropy `QTable` to pandas `DataFrame` - Loose all units info :(

In [ ]:
comet_table_pandas = QTable(comet_table).to_pandas()

In [ ]:
comet_table_pandas

### Can convert pandas `DataFrame` to astropy `QTable` - Will need to put the units back on :(

In [ ]:
comet_table_astropy = QTable.from_pandas(comet_table_pandas)

In [ ]:
comet_table_astropy

### Long Tables

In [ ]:
long_table= QTable.read('./Data/Comets_100.csv', format='ascii.csv')

In [ ]:
long_table

In [ ]:
long_table.show_in_notebook()